In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

# 读取训练集和测试集

In [11]:
dirPath = "data preprocessed - step 2"
trainSet_origin = pd.read_csv(dirPath + '\\train.csv')
testSet_origin = pd.read_csv(dirPath + '\\test.csv')

trainSet_origin.shape, testSet_origin.shape

((16637, 13), (742, 13))

In [12]:
trainSet_origin.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,1,2011/1/1,0,1,9.84,14.395,81,0,1,0,6,0,16
1,2,2011/1/1,1,1,9.02,13.635,80,0,1,0,6,0,40
2,3,2011/1/1,2,1,9.02,13.635,80,0,1,0,6,0,32
3,4,2011/1/1,3,1,9.84,14.395,75,0,1,0,6,0,13
4,5,2011/1/1,4,1,9.84,14.395,75,0,1,0,6,0,1


In [13]:
testSet_origin.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,16638,2012/12/1,0,1,10.66,15.150,81,0.0000,4,0,6,0,?
1,16639,2012/12/1,1,1,10.66,15.150,81,0.0000,4,0,6,0,?
2,16640,2012/12/1,2,2,10.66,15.150,81,0.0000,4,0,6,0,?
3,16641,2012/12/1,3,2,10.66,13.635,81,8.9981,4,0,6,0,?
4,16642,2012/12/1,4,1,10.66,14.395,81,6.0032,4,0,6,0,?


# 处理带有缺失值的数据

训练集中有些数据是用？来代替缺失值的，用excel查看后发现有22处缺失值，由于整个数据集比较大，所以把这些数据直接删除掉也无所谓。

In [14]:
trainSet_origin.columns
len(trainSet_origin.columns)

Index(['instant', 'dteday', 'hr', 'weathersit', 'temp', 'atemp', 'hum',
       'windspeed', 'season', 'holiday', 'weekday', 'workingday', 'cnt'],
      dtype='object')

13

In [24]:
t = trainSet_origin[(trainSet_origin.dteday == "?") |
                    (trainSet_origin.hr == "?") |
                    (trainSet_origin.weathersit == "?") |
                    (trainSet_origin.temp == "?") |
                    (trainSet_origin.atemp == "?") |
                    (trainSet_origin.hum == "?") |
                    (trainSet_origin.windspeed == "?")]

t
len(t)

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
218,219,2011/1/10,9,2,?,6.06,50,16.9979,1,0,1,1,94
304,305,2011/1/14,3,?,4.1,6.82,54,7.0015,1,0,5,1,1
355,356,2011/1/16,7,2,9.84,10.605,56,?,1,0,0,0,3
479,480,2011/1/22,0,1,1.64,?,45,16.9979,1,0,6,0,13
3664,3665,2011/6/6,14,?,31.98,34.09,31,8.9981,2,0,1,1,145
5248,5249,2011/8/11,14,1,32.8,34.85,?,0,3,0,4,1,142
6089,6090,2011/9/16,7,1,16.4,?,71,19.0012,3,0,5,1,299
8804,8805,2012/1/7,16,2,22.96,?,37,19.0012,1,0,6,0,401
9025,9026,2012/1/16,22,2,12.3,?,49,23.9994,1,1,1,0,43
9769,9770,2012/2/17,0,2,?,16.665,87,0,1,0,5,1,34


22

这里先选择将缺失值所在的行直接删除，后续再看看要不要修改吧。

In [25]:
trainSet_origin.shape
trainSet_origin = trainSet_origin.replace("?", np.NAN)
trainSet_origin = trainSet_origin.dropna()
trainSet_origin.shape

(16637, 13)

(16615, 13)

# 保存结果

In [26]:
dirPath = "data preprocessed - step 3"
if not os.path.exists(dirPath):
    os.makedirs(dirPath)

trainSet_origin.to_csv(dirPath + '\\train.csv', index=False, header=True)